In [1]:
import pandas as pd
import matplotlib.pyplot as plt

# Baca file Excel
df = pd.read_excel('raw_LOS.xlsx')  # Ganti dengan nama file Excel kamu

df.head()


,received_at_server,devUI,time_at_device,time_received_at_gateway,time_send_from_gateway,RSSI,protocol,frequency,packet_id,isNoise,jarak,jarak_rounded
0,2025-05-25 06:47:46,08:D1:F9:35:B5:48,2025-05-25 06:47:40,2025-05-25 06:47:45,2025-05-25 06:47:45.831,-35,mqtt,NaN,1,False,0.266667,0
1,2025-05-25 06:47:48,08:D1:F9:35:B5:48,2025-05-25 06:47:45,2025-05-25 06:47:47,2025-05-25 06:47:47.460,-29,mqtt,NaN,2,False,0.800000,1
2,2025-05-25 06:47:49,08:D1:F9:35:B5:48,2025-05-25 06:47:46,2025-05-25 06:47:48,2025-05-25 06:47:48.071,-29,mqtt,NaN,3,False,1.066667,1
3,2025-05-25 06:47:51,08:D1:F9:35:B5:48,2025-05-25 06:47:48,2025-05-25 06:47:50,2025-05-25 06:47:50.675,-30,mqtt,NaN,4,False,1.600000,2
4,2025-05-25 06:47:53,08:D1:F9:35:B5:48,2025-05-25 06:47:49,2025-05-25 06:47:51,2025-05-25 06:47:51.279,-37,mqtt,NaN,5,False,2.133333,2


In [2]:
df = df.drop(columns=['protocol', 'frequency'])

In [5]:
import pandas as pd
import numpy as np

# Konversi kolom waktu dengan handle invalid parsing
df['time_at_device'] = pd.to_datetime(df['time_at_device'], errors='coerce')
df['time_received_at_gateway'] = pd.to_datetime(df['time_received_at_gateway'], errors='coerce')

# Drop baris dengan waktu tidak valid (NaT)
valid_times_mask = df['time_at_device'].notna() & df['time_received_at_gateway'].notna()
df_valid = df[valid_times_mask].copy()

# Hitung delay (dalam detik) hanya untuk data valid
df_valid['delay'] = (df_valid['time_received_at_gateway'] - df_valid['time_at_device']).dt.total_seconds()

# Proses agregasi
df_processed = (
    df_valid.groupby('jarak_rounded', as_index=False)
    .agg({
        'RSSI': ['count', 'mean'],
        'delay': 'mean',
        'isNoise': 'sum'
    })
)

# Flatten multi-index columns
df_processed.columns = ['jarak_rounded', 'count', 'mean_RSSI', 'mean_delay', 'data_error']

# Hitung error rate
df_processed['error_rate'] = df_processed['data_error'] / df_processed['count']

# Filter kolom output
final_df = df_processed[[
    'jarak_rounded', 'count', 'mean_RSSI', 
    'mean_delay', 'data_error', 'error_rate'
]]

print("Data dengan waktu tidak valid diabaikan:")
print(f"Total baris asli: {len(df)} | Baris valid: {len(df_valid)}")
print(final_df)

Data dengan waktu tidak valid diabaikan:
Total baris asli: 1035 | Baris valid: 967
     jarak_rounded  count  mean_RSSI  mean_delay  data_error  error_rate
0                0      2     -45.50         3.5           0         0.0
1                1      4     -42.75         2.0           0         0.0
2                2      4     -40.75         2.0           0         0.0
3                3      2     -48.50         2.0           0         0.0
4                4      4     -50.00         2.0           0         0.0
..             ...    ...        ...         ...         ...         ...
388            455      1     -99.00         2.0           0         0.0
389            456      1    -101.00         2.0           0         0.0
390            467      1     -98.00         2.0           1         1.0
391            468      1     -97.00         2.0           0         0.0
392            506      2     -95.00         1.5           0         0.0

[393 rows x 6 columns]


In [6]:
output_file = "hasil_analisis_LOS.xlsx"
final_df.to_excel(output_file, index=False, sheet_name="Ringkasan")